In [ ]:
from __future__ import print_function
import numpy as np
import os
import time
import datetime
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, applications
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

from old_model import *
from new_model import * 

## Initialize GPU

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## Define plot functions and learning rate schedulers

In [ ]:
def annot_max(x,y, ax=None):
    """ Annotate Maximum values of currently open plot
    """
    xmax = x[np.argmax(y)]
    ymax = y.max()
    text= "x={:.3f}, y={:.3f}".format(xmax, ymax)
    if not ax:
        ax=plt.gca()
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    arrowprops=dict(arrowstyle="->",connectionstyle="angle,angleA=0,angleB=60")
    kw = dict(xycoords='data',textcoords="data",
              arrowprops=arrowprops, bbox=bbox_props, ha="left", va="top")
    ax.annotate(text, xy=(xmax, ymax), xytext=(xmax-.025, ymax-.025), **kw)
    
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 0.9*epoch:
        lr *= 0.5e-3
    elif epoch > 0.8*epoch:
        lr *= 1e-3
    elif epoch > 0.6*epoch:
        lr *= 1e-2
    elif epoch > 0.4*epoch:
        lr *= 1e-1
    else: 
        lr = 1e-3
    print('Learning rate: ', lr)
    return lr

## Define Models

In [ ]:
def cifarFPNRes50e100(image_shape, class_number):
    """
        Single output FPN fed to a Dense layer to accommodate recognition with Cifar-10.
        Uses ResNet50 as backbone
    """
    resnet50Backbone = get_backbone_ResNet50(input_shape=image_shape)
    model = customFeaturePyramid2(resnet50Backbone, class_number)
    return model

def cifarFPNRes50V2e100(image_shape, class_number):
    resnet50V2Backbone = get_backbone_ResNet50V2(input_shape=image_shape)
    model = customFeaturePyramid2(resnet50V2Backbone, class_number)
    return model

def cifarFPNRes101e200(image_shape, class_number):
    resnet101Backbone = get_backbone_ResNet101(input_shape=image_shape)
    model = customFeaturePyramid2(resnet101Backbone, class_number)
    return model

## Data preprocessing 

In [ ]:
#class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
#               'dog', 'frog', 'horse', 'ship', 'truck']

batch_size = 16  # orig paper trained all networks with batch_size=128
epochs = 200
########## Specify models to train and test with cifar-10 here
model_list = [cifarFPNRes50e100, cifarFPNRes101e200]
num_classes = 10

(train_img, train_lab), (test_img, test_lab) = datasets.cifar10.load_data() #load dataset
train_img, test_img = train_img/255.0, test_img/255.0 #normalize dataset
# convert label to binary encoding
train_lab = keras.utils.to_categorical(train_lab, num_classes)
test_lab = keras.utils.to_categorical(test_lab, num_classes)
val_img  = train_img[40000:]
val_lab = train_lab[40000:]
train_img = train_img[0:40000]
train_lab = train_lab[0:40000]
# print(train_img.shape,test_img.shape)
img_shape = train_img.shape[1:]
img_rows = train_img[0].shape[0]
img_cols = train_img[0].shape[1]

In [ ]:
test_loss = []
test_acc = []
time_taken = []
# opt = 'adam'
# loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = Adam(learning_rate = lr_schedule(epochs))
loss = 'categorical_crossentropy'

for f in model_list:
    model = f(img_shape, num_classes)
#     model.build(img_shape)
#     model.summary()
    model.compile(optimizer=opt,
                  loss=loss,
                  metrics=['accuracy'])

    history = model.fit(train_img,
                            train_lab,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(test_img, test_lab),
                            shuffle=True)
    ########## save model
    #https://stackoverflow.com/questions/51806852/cant-save-custom-subclassed-model
    model.save_weights('{}_epoch{}_batch{}_weights'.format(f.__name__, epochs, batch_size), save_format='tf')
    
    ########## plot stuff
    plt.style.use('seaborn')
    x = [i for i in range(1, epochs+1)]
    fig = plt.figure()
    ########## plot model training accuracy
    plt.subplot(211)
    acc = np.array(history.history['accuracy'])
    valAcc = np.array(history.history['val_accuracy'])
    plt.plot(acc)
    plt.plot(valAcc)
    annot_max(x, acc)
    annot_max(x, valAcc)
    plt.legend(['train', 'test'], loc='lower right')
    plt.title('{}_trainAcc'.format(f.__name__))
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    ########## plot model training loss
    plt.subplot(212)
    loss = np.array(history.history['loss'])
    valLoss = np.array(history.history['val_loss'])
    plt.plot(loss)
    plt.plot(valLoss)
    annot_max(x, loss)
    annot_max(x, valLoss)
    plt.legend(['train', 'test'], loc='lower right')
    plt.title('{}_trainLoss'.format(f.__name__))
    plt.ylabel('loss')
    plt.xlabel('epoch')

    plt.savefig('{}_trainAccLoss'.format(f.__name__), format='png')
    plt.close(plt.gcf())

    time1 = time.time()
    a, b = model.evaluate(test_img, test_lab, verbose=2)
    time_taken.append(time.time() - time1)
    test_loss.append(a)
    test_acc.append(b)

print(test_loss)
print(test_acc)
print(time_taken)

In [ ]:
model.summary()